In [ ]:
# 필수 라이브러리 임포트
import pandas as pd
import pandas_datareader.wb as wb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import numpy as np

# 분석 기간 설정 (2000-2024)
START_YEAR = 2000
END_YEAR = 2024


In [ ]:
# # # 세계은행 데이터 수집
# # indicators = {
# #     # 'EG.ELC.ACCS.ZS': 'Electricity_Access',
# #     # 'IT.CEL.SETS.P2': 'Mobile_Subscriptions',
# #     # 'IT.NET.USER.ZS': 'Internet_Users',
# #     'IS.RRS.TOTL.KM': 'Railway_Length',
# #     'NY.GDP.MKTP.KD.ZG': 'GDP_Growth'
# # }

# # df = wb.download(indicator=list(indicators.keys()), 
# #                 country = ['UKR', 'RUS', 'AFG', 'IRQ', 'ISR', 'PSE', 'SYR', 'MLI'], 
# #                 start=START_YEAR, 
# #                 end=END_YEAR).reset_index()

# # # 컬럼명 변경 및 전처리
# # df = df.rename(columns=indicators)
# # df['Year'] = df['year'].astype(int)
# # df = df.dropna().set_index('Year')

import pandas as pd
import pandas_datareader.wb as wb
import time

# 세계은행 데이터 수집
indicators = {
    # 'EG.ELC.ACCS.ZS': 'Electricity_Access',
    # 'IT.CEL.SETS.P2': 'Mobile_Subscriptions',
    # 'IT.NET.USER.ZS': 'Internet_Users',
    # 'IS.RRS.TOTL.KM': 'Railway_Length',
    'NY.GDP.MKTP.KD.ZG': 'GDP_Growth'
}

countries = ['UKR', 'RUS', 'AFG', 'IRQ', 'ISR', 'PSE', 'SYR', 'MLI']

# 데이터프레임을 저장할 리스트
all_data = []

# 국가를 작은 그룹으로 나누기
country_groups = [countries[i:i+2] for i in range(0, len(countries), 2)]

for country_group in country_groups:
    try:
        print(f"데이터 요청 중: {country_group}")
        temp_df = wb.download(
            indicator=list(indicators.keys()), 
            country=country_group, 
            start=START_YEAR, 
            end=END_YEAR
        )
        if not temp_df.empty:
            all_data.append(temp_df)
        # API 서버에 부담을 줄이기 위한 대기 시간
        time.sleep(2)
    except Exception as e:
        print(f"국가 {country_group}에 대한 데이터 요청 실패: {e}")

# 모든 데이터프레임 합치기
if all_data:
    df = pd.concat(all_data).reset_index()
    # 컬럼명 변경 및 전처리
    df = df.rename(columns=indicators)
    print("데이터 수집 완료!")
else:
    print("데이터를 가져오지 못했습니다.")


In [ ]:
df

In [ ]:
df = df.drop('country', axis=1)

In [ ]:
# 복구 비용 계산
df['Recovery_Cost_Index'] = (
    (df['GDP_Growth']/100 * df['Mobile_Subscriptions']) + 
    (df['Electricity_Access'] * df['Railway_Length'])
)

# 데이터 정규화
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), 
                        columns=df.columns, 
                        index=df.index)


In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Infrastructure Indicators Correlation Analysis for Iraq (2003-2023)')
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 12))

# Electricity Access vs Recovery Cost
axes[0].plot(df.index, df['Electricity_Access'], label='Electricity Access')
axes[0].plot(df.index, df['Recovery_Cost_Index'], label='Recovery Cost Index')
axes[0].set_title('Energy Infrastructure Recovery Trends')

# Mobile Subscriptions vs GDP Growth
axes[1].scatter(df['Mobile_Subscriptions'], df['GDP_Growth'], 
               c=df.index, cmap='viridis')
axes[1].set_xlabel('Mobile Subscriptions per 100 people')
axes[1].set_ylabel('GDP Growth (%)')

# Railway Infrastructure Investment Effect
axes[2].bar(df.index, df['Railway_Length'], 
           color=np.where(df['Railway_Length']>0, 'green', 'red'))
axes[2].set_title('Annual Railway Infrastructure Expansion')

plt.tight_layout()
plt.show()
